In [2]:
import pandas as pd
import numpy as np

## Fig4a

In [3]:
all_DGV = pd.read_csv('../../map_drugs/all_DGVs.txt',sep='\t')

In [4]:
all_DGV = all_DGV[all_DGV['path'].isnull()].drop_duplicates(['ChEMBL','Host Gene'])

In [5]:
all_DGV = all_DGV[['drug_claim_primary_name','ChEMBL','interaction_type','gene_name','Direction','Clinical_phase','Virus','D2V','Host Gene Type', 'Host Gene Source', 'G2V',
       'Direciton Type', 'Direction Source',]]

In [6]:
all_DGV = all_DGV[(all_DGV['Clinical_phase']>=0)&(all_DGV['Virus']=='SARS-CoV2')]

In [7]:
drugs_rank = pd.read_csv('../../map_drugs/drugs_rank.csv')
candidate_drugs = drugs_rank[drugs_rank['Rating Score']>=-10]['CHEMBL ID'].unique()
clinicaltrial = pd.read_csv('../../map_drugs/clinicaltrial.csv')
clinicaltrial_all = clinicaltrial[(clinicaltrial['ChEMBL'].isin(drugs_rank['CHEMBL ID'].values))&(clinicaltrial['Clinical_phase']>=0)]['ChEMBL'].unique()

In [8]:
for gene,sub_df in all_DGV.groupby('gene_name'):
    if len(sub_df['Direction'].unique()) == 2:
        if len(sub_df[sub_df['ChEMBL'].isin(clinicaltrial_all)]) >0:
            print(gene)
            print(sub_df['Direction'].value_counts())
            print(sub_df['Virus'].unique())

ATP2C1
-1.0    5
 1.0    1
Name: Direction, dtype: int64
['SARS-CoV2']
CCR5
-1.0    21
 1.0     1
Name: Direction, dtype: int64
['SARS-CoV2']
PTGER4
 1.0    19
-1.0    10
Name: Direction, dtype: int64
['SARS-CoV2']
SIGMAR1
 1.0    13
-1.0     5
Name: Direction, dtype: int64
['SARS-CoV2']


In [14]:
all_DGV[(all_DGV['gene_name']=='SIGMAR1')&(all_DGV['ChEMBL'].isin(clinicaltrial_all))]

,drug_claim_primary_name,ChEMBL,interaction_type,gene_name,Direction,Clinical_phase,Virus,D2V,Host Gene Type,Host Gene Source,G2V,Direciton Type,Direction Source
281887,Naltrexone,CHEMBL19019,antagonist,SIGMAR1,-1.0,0.0,SARS-CoV2,1.0,PPI,"2020.06.17.156455,j.medj.2020.07.002",-1,DEG,GSE152586
281896,FLUVOXAMINE,CHEMBL814,antagonist,SIGMAR1,-1.0,0.0,SARS-CoV2,1.0,PPI,"2020.06.17.156455,j.medj.2020.07.002",-1,DEG,GSE152586


In [9]:
fig4a = all_DGV[(all_DGV['gene_name']=='SIGMAR1')&(all_DGV['Clinical_phase']==4)].copy()
fig4a['G2V'] = fig4a['G2V']*-1

In [10]:
fig4a

,drug_claim_primary_name,ChEMBL,interaction_type,gene_name,Direction,Clinical_phase,Virus,D2V,Host Gene Type,Host Gene Source,G2V,Direciton Type,Direction Source


In [262]:
fig4a_edge = pd.DataFrame(columns=['source', 'target', 'direction', 'mapping'])
ith = 0
for drug, gene, effct in fig4a[[
        'drug_claim_primary_name', 'gene_name', 'Direction'
]].values:
    fig4a_edge.loc[ith] = drug, gene, effct, 'direct'
    ith += 1
for gene, virus, effct in fig4a[[
        'gene_name', 'Virus', 'G2V'
]].values:
    fig4a_edge.loc[ith] = gene, virus, effct, 'direct'
    ith += 1
fig4a_edge.drop_duplicates().to_excel('./fig4a_edge_approved.xlsx',index=False)

In [268]:
fig4a_node = pd.DataFrame(
    columns=['name', 'type', 'virus', 'max_phase', 'trial'])
ith = 0
for drug, virus, max_phase in fig4a[[
        'drug_claim_primary_name', 'Virus', 'Clinical_phase'
]].values:
    fig4a_node.loc[ith] = drug, 'drug', virus, max_phase, 1
    ith += 1
for gene, virus, max_phase in fig4a[[
        'gene_name', 'Virus', 'Clinical_phase'
]].values:
    fig4a_node.loc[ith] = gene, 'gene', virus, np.nan, np.nan
    ith += 1
for i, virus in enumerate(['SARS-CoV2', 'SARS-CoV', 'MERS-CoV'], start=1):
    fig4a_node.loc[ith] = virus, f'virus{i}', np.nan, np.nan, np.nan
    ith += 1
fig4a_node.drop_duplicates().to_excel('./fig4a_node_approved.xlsx',index=False)

## Fig 4b

In [15]:
anno_reposition = pd.read_csv('../../map_drugs/anno_repostition.txt',sep='\t')
side_cormo = pd.read_csv('../../map_drugs/side_cormo.txt', sep='\t')

In [278]:
anno_reposition = anno_reposition[anno_reposition['Path'].isnull()]

In [8]:
drugs_rank = pd.read_csv('../../map_drugs/drugs_rank.csv')
candidate_drugs = drugs_rank[drugs_rank['Rating Score']>=-10]['CHEMBL ID'].unique()
clinicaltrial = pd.read_csv('../../map_drugs/clinicaltrial.csv')
clinicaltrial_all = clinicaltrial[(clinicaltrial['ChEMBL'].isin(drugs_rank['CHEMBL ID'].values))&(clinicaltrial['Clinical_phase']>=0)]['ChEMBL'].unique()

In [22]:
anno_reposition[(anno_reposition['Path'].isnull())&(anno_reposition['CHEMBL ID']=='CHEMBL190461')]

,Drug,Virus,Host Gene,Host Gene Type,Host Gene Source,Direction,Direction Type,Direction Source,Drug Target,DTI Type,DTI Score,Drug Effect,Path,RWR Prob,CHEMBL ID,Clinical Phase,fake score
486,Cannabidiol,SARS-CoV2,GPX1,PPI,"32353859,33060197",-1,DEG,GSE152586,GPX1,stimulator,0.88,1.0,NaN,1.0,CHEMBL190461,4.0,10000.0


In [26]:
anno_reposition[anno_reposition['CHEMBL ID']=='CHEMBL71']

,Drug,Virus,Host Gene,Host Gene Type,Host Gene Source,Direction,Direction Type,Direction Source,Drug Target,DTI Type,DTI Score,Drug Effect,Path,RWR Prob,CHEMBL ID,Clinical Phase,fake score
423,Chlorpromazine,SARS-CoV,KCNH2,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,KCNH2,inhibitor,0.01,-1.0,NaN,1.000000,CHEMBL71,4.0,10000.00
33882,Chlorpromazine,MERS-CoV,MMP9,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,SMPD1,inhibitor,0.57,-1.0,6609->1950->4318,0.000223,CHEMBL71,4.0,0.57
33885,Chlorpromazine,SARS-CoV2,PPP2R2A,CRISPR-Screening,33382968,1,CRISPR-Screening,33382968,SMPD1,inhibitor,0.57,-1.0,6609->1950->5520,0.000135,CHEMBL71,4.0,0.57
33886,Chlorpromazine,SARS-CoV2,PPP2CA,CRISPR-Screening,33382968,1,CRISPR-Screening,33382968,SMPD1,inhibitor,0.57,-1.0,6609->1950->5515,0.000141,CHEMBL71,4.0,0.57
33887,Chlorpromazine,SARS-CoV2,CSK,CRISPR-Screening,33382968,1,CRISPR-Screening,33382968,SMPD1,inhibitor,0.57,-1.0,6609->1950->1445,0.000150,CHEMBL71,4.0,0.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97539,CHLORPROMAZINE,SARS-CoV2,STX1A,CRISPR-Screening,33333024,-1,CRISPR-Screening,33333024,ADRA2C,antagonist,0.01,-1.0,152->2771-|6804,0.000131,CHEMBL71,4.0,0.01
97546,CHLORPROMAZINE,SARS-CoV2,HLA-DRB1,GWAS,GRASP,-1,eQTL,gtex_v8,ADRA2C,antagonist,0.01,-1.0,152->2775-|3123,0.000125,CHEMBL71,4.0,0.01
97557,CHLORPROMAZINE,SARS-CoV2,GNGT2,GWAS,GRASP,1,eQTL,gtex_v8,ADRA2C,antagonist,0.01,-1.0,152->2793,0.006819,CHEMBL71,4.0,0.01
97586,CHLORPROMAZINE,SARS-CoV,CNGA3,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,ADRA2C,antagonist,0.01,-1.0,152->2780->1261,0.000144,CHEMBL71,4.0,0.01


In [20]:
side_cormo[side_cormo['ChEMBL']=='CHEMBL190461'][['drug_claim_primary_name','ChEMBL','interaction_type','Direction','gene_name_x','Disease','DirCE','Clinical_phase']]

,drug_claim_primary_name,ChEMBL,interaction_type,Direction,gene_name_x,Disease,DirCE,Clinical_phase
1421,Cannabidiol,CHEMBL190461,activator,1.0,PPARG,"Diabetes Mellitus, Type 2",I,0.0
1503,Cannabidiol,CHEMBL190461,activator,1.0,PPARG,"Diabetes Mellitus, Type 2",I,0.0
6643,Cannabidiol,CHEMBL190461,stimulator,1.0,GPX1,Coronary Artery Disease,I,0.0


In [19]:
side_cormo.columns

Index(['drug_claim_primary_name', 'drug_concept_id', 'gene_name_x',
       'entrez_id', 'interaction_type', 'interaction_claim_source',
       'interaction_group_score', 'Direction', 'ChEMBL_guess', 'ChEMBL',
       'Clinical_phase', 'source', 'target', 'prob', 'path', 'G2G', 'Disease',
       'gwas id', 'rsid', 'ref', 'alt', 'pos', 'risk', 'GWAS P', 'GWAS Beta',
       'Finemapping Pro', 'gene id', 'gene_name_y', 'DirCE', 'type',
       'eQTL/BSIFT', 'entrez_disease', 'G2V', 'D2V'],
      dtype='object')

## fig4c

In [3]:
anno_reposition = pd.read_csv('../../map_drugs/anno_repostition.txt',sep='\t')
side_cormo = pd.read_csv('../../map_drugs/side_cormo.txt', sep='\t')
side_cormo = side_cormo[side_cormo['ChEMBL'].isin(anno_reposition['CHEMBL ID'].values)]

In [4]:
side_effect = side_cormo[side_cormo['D2V']==1]
side_effect_map = pd.Series(dtype=str)
side_effect_counts = pd.Series(dtype=int)
for drug, sub_df in side_effect.groupby('drug_claim_primary_name'):
    diseases = sub_df['Disease'].unique()
    side_effect_map[drug] = ';'.join(diseases)
    side_effect_counts[drug] = len(diseases)

In [386]:
anno_reposition = anno_reposition[anno_reposition['Path'].isnull()]

In [5]:
anno_reposition['Side Effect'] = anno_reposition['Drug'].map(side_effect_map).fillna('')
anno_reposition['Side Effect Counts'] = anno_reposition['Drug'].map(side_effect_counts).fillna(0)

In [6]:
anno_reposition = anno_reposition[anno_reposition['Side Effect Counts']>0]

In [11]:
anno_reposition[anno_reposition['CHEMBL ID'].isin(clinicaltrial_all)].to_csv('./a.csv')

In [7]:
anno_reposition.sort_values('Side Effect Counts')[:30]

,Drug,Virus,Host Gene,Host Gene Type,Host Gene Source,Direction,Direction Type,Direction Source,Drug Target,DTI Type,DTI Score,Drug Effect,Path,RWR Prob,CHEMBL ID,Clinical Phase,fake score,Side Effect,Side Effect Counts
81203,VORUCICLIB,SARS-CoV2,TP53,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,CDK5,inhibitor,0.08,-1.0,1020->7157,0.007020,CHEMBL3545218,0.0,0.08,Coronary Artery Disease,1.0
74379,Purvalanol,SARS-CoV,ERBB3,PPI,2020.06.17.156455,1,DEG,GSE33267,MAPK1,inhibitor,0.11,-1.0,5594->4233->2065,0.000182,CHEMBL23254,0.0,0.11,Multiple Sclerosis,1.0
74374,Ranolazine,SARS-CoV2,PIK3C3,CRISPR-Screening,33147445,1,CRISPR-Screening,33147445,ADRA1A,antagonist,0.11,-1.0,148->5562->5289,0.000597,CHEMBL1404,4.0,0.11,Essential Hypertension,1.0
74369,AZD-5438,SARS-CoV2,TP53,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,CDK9,inhibitor,0.11,-1.0,1025->472->7157,0.000700,CHEMBL488436,1.0,0.11,Essential Hypertension,1.0
74360,Arotinolol,MERS-CoV,AMBRA1,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,ADRA1A,antagonist,0.11,-1.0,148->5562->55626,0.000505,CHEMBL93298,3.0,0.11,Essential Hypertension,1.0
74348,Purvalanol,SARS-CoV2,MAPKAPK3,CRISPR-Screening,33686287,1,CRISPR-Screening,33686287,MAPK1,inhibitor,0.11,-1.0,5594->7867,0.000754,CHEMBL23254,0.0,0.11,Multiple Sclerosis,1.0
74346,Ranolazine,MERS-CoV,AMBRA1,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,ADRA1A,antagonist,0.11,-1.0,148->5562->55626,0.000505,CHEMBL1404,4.0,0.11,Essential Hypertension,1.0
74335,"ESTROGENS, CONJUGATED SYNTHETIC A",SARS-CoV,MET,PPI,2020.06.17.156455,-1,DEG,GSE33267,ESR1,agonist,0.11,1.0,2099->1950->4233,0.000125,CHEMBL1201466,4.0,0.11,Atrial Fibrillation,1.0
74334,"SYNTHETIC CONJUGATED ESTROGENS, B",SARS-CoV,MET,PPI,2020.06.17.156455,-1,DEG,GSE33267,ESR1,agonist,0.11,1.0,2099->1950->4233,0.000125,CHEMBL1201467,4.0,0.11,Atrial Fibrillation,1.0
74331,"SYNTHETIC CONJUGATED ESTROGENS, B",SARS-CoV2,STK11,CRISPR-Screening,33147444,1,CRISPR-Screening,33147444,ESR1,agonist,0.11,1.0,2099-|6794,0.000901,CHEMBL1201467,4.0,0.11,Atrial Fibrillation,1.0


In [13]:
side = side_cormo[side_cormo['D2V']==1][['drug_claim_primary_name','ChEMBL','interaction_type','Direction','gene_name_x','Disease','DirCE','Clinical_phase']].copy()

In [14]:
side = side.drop_duplicates()

In [362]:
side['gene_name_x'].unique()

array(['SREBF1', 'CLCN6', 'TYK2', 'NGF', 'CALCRL', 'PPARG', 'MAPK3',
       'ARG1', 'BLK', 'FCGR2A', 'ADRB1', 'MC1R', 'CRHR1', 'PROCR',
       'CHRNB1', 'ITPR3', 'FES', 'VKORC1', 'CDK10', 'MCL1', 'CTSK',
       'CASP8', 'SIRT1', 'GGCX', 'ESR2', 'TESK2', 'KCNN4', 'CSNK1G1',
       'SLC22A1', 'DAGLB', 'LPL', 'APOE', 'P2RX4', 'ALDH2', 'ENPEP'],
      dtype=object)

In [20]:
side[side['ChEMBL']=='CHEMBL1200973']

,drug_claim_primary_name,ChEMBL,interaction_type,Direction,gene_name_x,Disease,DirCE,Clinical_phase
6115,Estradiol cypionate,CHEMBL1200973,agonist,1.0,ESR2,Atrial Fibrillation,A,0.0


In [363]:
side[side['Direction']==1]['gene_name_x'].unique()

array(['NGF', 'CALCRL', 'FCGR2A', 'MC1R', 'CHRNB1', 'ITPR3', 'SIRT1',
       'GGCX', 'ESR2', 'KCNN4', 'P2RX4'], dtype=object)

In [364]:
side_cormo[(side_cormo['gene_name_x']=='TYK2')&(side_cormo['Direction']==1)]

,drug_claim_primary_name,drug_concept_id,gene_name_x,entrez_id,interaction_type,interaction_claim_source,interaction_group_score,Direction,ChEMBL_guess,ChEMBL,...,GWAS Beta,Finemapping Pro,gene id,gene_name_y,DirCE,type,eQTL/BSIFT,entrez_disease,G2V,D2V


In [373]:
side[side['gene_name_x']=='ITPR3']

,drug_claim_primary_name,ChEMBL,interaction_type,Direction,gene_name_x,Disease,DirCE,Clinical_phase
4876,IP3,CHEMBL279107,activator,1.0,ITPR3,Multiple Sclerosis,A,0.0


In [399]:
side[side['ChEMBL'].isin(clinicaltrial_all)]

,drug_claim_primary_name,ChEMBL,interaction_type,Direction,gene_name_x,Disease,DirCE,Clinical_phase
613,TOFACITINIB,CHEMBL221959,inhibitor,-1.0,TYK2,"Liver Cirrhosis, Biliary",I,0.0
628,RUXOLITINIB,CHEMBL1789941,inhibitor,-1.0,TYK2,"Liver Cirrhosis, Biliary",I,0.0
644,TOFACITINIB,CHEMBL221959,inhibitor,-1.0,TYK2,"Lupus Erythematosus, Systemic",I,0.0
659,RUXOLITINIB,CHEMBL1789941,inhibitor,-1.0,TYK2,"Lupus Erythematosus, Systemic",I,0.0
861,TOFACITINIB,CHEMBL221959,inhibitor,-1.0,TYK2,"Diabetes Mellitus, Type 1",I,0.0
876,RUXOLITINIB,CHEMBL1789941,inhibitor,-1.0,TYK2,"Diabetes Mellitus, Type 1",I,0.0
1412,CDDO-ME,CHEMBL1762621,antagonist,-1.0,PPARG,"Diabetes Mellitus, Type 2",I,0.0
1425,RESVERATROL,CHEMBL165,antagonist,-1.0,PPARG,"Diabetes Mellitus, Type 2",I,0.0
2244,Nebivolol,CHEMBL434394,antagonist,-1.0,ADRB1,Essential Hypertension,I,0.0
6115,Estradiol cypionate,CHEMBL1200973,agonist,1.0,ESR2,Atrial Fibrillation,A,0.0


In [403]:
anno_reposition[anno_reposition['Drug']=='Estradiol cypionate']

,Drug,Virus,Host Gene,Host Gene Type,Host Gene Source,Direction,Direction Type,Direction Source,Drug Target,DTI Type,DTI Score,Drug Effect,Path,RWR Prob,CHEMBL ID,Clinical Phase,fake score
101104,Estradiol cypionate,MERS-CoV,NOS2,CRISPR-Screening,33147444,-1,CRISPR-Screening,33147444,ESR2,agonist,0.0,1.0,2100->5600->4843,0.000092,CHEMBL1200973,4.0,0.0
101190,Estradiol cypionate,MERS-CoV,EGLN1,CRISPR-Screening,33147444,-1,CRISPR-Screening,33147444,ESR2,agonist,0.0,1.0,2100->54583,0.002309,CHEMBL1200973,4.0,0.0
101580,Estradiol cypionate,SARS-CoV2,UBC,CRISPR-Screening,33147445,1,CRISPR-Screening,33147445,ESR2,agonist,0.0,1.0,2100->1950-|7316,0.000383,CHEMBL1200973,4.0,0.0
101797,Estradiol cypionate,SARS-CoV2,PLCB4,CRISPR-Screening,33686287,-1,CRISPR-Screening,33686287,ESR2,agonist,0.0,1.0,2100->2776->5332,0.000148,CHEMBL1200973,4.0,0.0
101853,Estradiol cypionate,SARS-CoV2,HK2,CRISPR-Screening,33382968,-1,CRISPR-Screening,33382968,ESR2,agonist,0.0,1.0,2100->4609->3099,0.000305,CHEMBL1200973,4.0,0.0
102160,Estradiol cypionate,SARS-CoV,EPHA2,PPI,2020.06.17.156455,-1,DEG,GSE33267,ESR2,agonist,0.0,1.0,2100->1950->1969,0.000272,CHEMBL1200973,4.0,0.0
102303,Estradiol cypionate,SARS-CoV,NTRK3,CRISPR-Screening,33147444,-1,CRISPR-Screening,33147444,ESR2,agonist,0.0,1.0,2100->1950->4916,0.000239,CHEMBL1200973,4.0,0.0
102497,Estradiol cypionate,MERS-CoV,TP53,PPI,33060197,-1,DEG,GSE122876,ESR2,agonist,0.0,1.0,2100->2353->7157,0.000167,CHEMBL1200973,4.0,0.0
126443,Estradiol cypionate,SARS-CoV,NUP98,PPI,33060197,-1,DEG,GSE33267,ESR2,agonist,0.0,1.0,2100->4928,0.002370,CHEMBL1200973,4.0,0.0
126543,Estradiol cypionate,SARS-CoV,JUN,PPI,STRING Viruses,-1,DEG,GSE33267,ESR2,agonist,0.0,1.0,2100->3725,0.002374,CHEMBL1200973,4.0,0.0


In [ ]:
anno_reposition